**1. Import Dependencies**

In [ ]:
# Import Gym Stuff
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# Import helpers
import numpy as np
import random
import os

# Import stable baselines stuff
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv 
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
#Import Game Stuff
from snakeGame import SnakeGame

**2. Building the ENV**

In [ ]:
class SnakeENV(Env):
    def __init__(self) -> None:
        self.game = SnakeGame()

        self.action_space = Discrete(4)
        self.observation_space = Dict(['head_pos', Box(low=np.array([0, 0]), high=np.array([self.game.window.width, self.game.window.height])),
                                        'fruit_pos', Box(low=np.array([0, 0]), high=np.array([self.game.window.width, self.game.window.height])),
                                        'length', Box(low=np.array([0]), high=np.array([np.inf])),
                                        'direction', Discrete(4)
                                ])


    def step(self, action):
        direction = self.action_to_direction(action)
        reward, done = self.game.main(direction)

        info = {}

        return self.state, reward, done, info

    def render(self):
        pass

    def reset(self):
        self.game = SnakeGame()
        self.state = {'head_pos': np.array(self.game.snake.head_pos).astype(int),
                        'fruit_pos': np.array(self.game.fruit.pos).astype(int),
                        'lenght': np.array(len(self.game.snake.body)).astype(int),
                        'direction': self.direction_to_action(self.game.snake.direction)
                    }
        return self.state


    def action_to_direction(self, action):
        if action == 0:
            return 'UP'
        elif action == 1:
            return 'DOWN'
        elif action == 2:
            return 'LEFT'
        elif action == 3:
            return 'RIGHT'
    def direction_to_action(self, direction):
        if direction == 'UP':
            return 0
        elif direction == 'DOWN':
            return 1
        elif direction == 'LEFT':
            return 2
        elif direction == 'RIGHT':
            return 3
